In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import tqdm as tqdm
from IPython.core.display import display
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load data
cust_demand = pd.read_csv('data/customer_demand.csv')
sku_wt_vol = pd.read_csv('data/sku_weight_volume.csv')
fixed_cost = pd.read_csv('data/fixed_cost.csv')
var_cost = pd.read_csv('data/variable_cost.csv')
capacity = pd.read_csv('data/capacity.csv')
site_loc = pd.read_csv('data/site_lat_long.csv')

In [ ]:
# extract customer and dc locations
## customer location
cust_loc = site_loc[site_loc['site'].str.startswith('C')] # extract data with string starting 'C'
cust_loc.reset_index() # reset index
cust_loc.to_csv('extracted_data/1_cust_loc.csv', index=False) # extract csv file

## dc location
dc_loc = site_loc[site_loc['site'].str.startswith('DC')] # extract data with string starting 'DC'
dc_loc.reset_index() # reset index
dc_loc.to_csv('extracted_data/2_dc_loc.csv', index=False) # extract csv file

In [ ]:
since = time.time()
# calculate the unit_transportation cost
## calculate sqrt(wt_times_vol) / 20060 (group_num 6*10 + 20,000) and let's call it multiplication_factor_01(mf_01).
sku_wt_vol['mf_01'] = np.sqrt(sku_wt_vol['weight'] * sku_wt_vol['volume']) / 20060
sku_wt_vol.to_csv('extracted_data/31_sku_wt_vol_mf01.csv') # extract csv file


## calculate sqrt((𝑙𝑎𝑡𝑖 − 𝑙𝑎𝑡𝑗)2 + (𝑙𝑜𝑛𝑔𝑖 − 𝑙𝑜𝑛𝑔𝑗)2) and call it multiplication_factor_02 (mf_02)
### use cust and dc location datasets
### convert pandas dataframe to numpy array
### Iterate over data and save enteries in list as extracted data
### finally convert list in dictionary, then pandas dataframe and save it as csv file
dc_loc_arr = dc_loc.values
cust_loc_arr = cust_loc.values
dc = []
cust = []
mf_02 = []
for i in range (len(dc_loc_arr)):
    for j in range(len(cust_loc_arr)):
        dc.append(dc_loc_arr[i][0])
        cust.append(cust_loc_arr[j][0])
        mf02_calc = np.sqrt(((dc_loc_arr[i][1] - cust_loc_arr[j][1])**2) + ((dc_loc_arr[i][2] - cust_loc_arr[j][2])**2))
        mf_02.append(mf02_calc)

dc_cust_mf02 = {'dc': dc, 'cust': cust, 'mf_02': mf_02}
dc_cust_mf02 = pd.DataFrame(dc_cust_mf02)
dc_cust_mf02.to_csv('extracted_data/32_dc_cust_mf02.csv') # extract csv file


## link dc_cust_sku dataframes
### convert those to arr 
### iterate sku array over dc and cust arrays and save enteries in list as extracted data
### finally conver list in dictionary, then pandas dataframe but don't export it - too big to export csv.
dc_cust_mf02_arr = dc_cust_mf02.values
sku_wt_vol_arr = sku_wt_vol.values
dc = []
cust = []
sku = []
unit_trans_cost = []
for i in range(len(dc_cust_mf02_arr)):
    for j in range (len(sku_wt_vol_arr)):
        dc.append(dc_cust_mf02_arr[i][0])
        cust.append(dc_cust_mf02_arr[i][1])
        sku.append(sku_wt_vol_arr[j][0])
        unit_trans_cost_calc = dc_cust_mf02_arr[i][2] * sku_wt_vol_arr[j][4]
        unit_trans_cost.append(unit_trans_cost_calc)
dc_cust_sku_unit_trans_cost = {'dc': dc, 'cust': cust, 'sku': sku, 'unit_trans_cost': unit_trans_cost}
dc_cust_sku_unit_trans_cost = pd.DataFrame(dc_cust_sku_unit_trans_cost)
dc_cust_sku_unit_trans_cost.head()

# extract the minimum unit transportation cost
dc_cust_sku_unit_trans_cost_grpd = dc_cust_sku_unit_trans_cost.groupby(['cust', 'sku'])['unit_trans_cost'].min() # group data by cust and skus, and find minimum unit transportation cost based on cust and sku
dc_cust_sku_unit_trans_cost_idx = dc_cust_sku_unit_trans_cost.set_index(['cust', 'sku']) # set index on original dataframe
dc_cust_sku_min_unit_trans_cost = pd.merge(dc_cust_sku_unit_trans_cost_grpd, dc_cust_sku_unit_trans_cost_idx, on=['cust', 'sku', 'unit_trans_cost'], how='left') # merge grouped and indexed dataframes
print('check null values in dc_cust_sku_min_unit_trans_cost')
print(pd.isna(dc_cust_sku_min_unit_trans_cost).sum()) # check whether there are any null values in extracted dataframe
dc_cust_sku_min_unit_trans_cost.to_csv('extracted_data/33_dc_cust_sku_min_unit_trans_cost.csv') # extract csv file


# Merge customer demand with unit price calculations to calculate transportation cost
cust_demand.rename(columns={'customer': 'cust', 'product': 'sku'}, inplace=True) # rename columns to match with other dataframe to merge easily
cust_demand_idx = cust_demand.set_index(['cust', 'sku']) # set index
dc_cust_total_transport_cost = pd.merge(cust_demand_idx, dc_cust_sku_min_unit_trans_cost, on=['cust', 'sku'], how='left') # merge dataframes
dc_cust_total_transport_cost.reset_index(['cust', 'sku'], inplace=True) # resetting indices
dc_cust_total_transport_cost['total_trans_cost'] = dc_cust_total_transport_cost['quantity'] * dc_cust_total_transport_cost['unit_trans_cost'] # calculate transportation cost
dc_cust_total_transport_cost[['dc', 'cust', 'sku', 'quantity', 'unit_trans_cost', 'total_trans_cost']] # re-arrange column sequence
dc_cust_total_transport_cost.dropna(inplace=True) # drop any null values
print('check null values in dc_cust_total_transportation_cost')
print(pd.isna(dc_cust_total_transport_cost).sum()) # check whether there are any null values in extracted dataframe
print('Shape of dataframe - dc_cust_total_transportation_cost:', dc_cust_total_transport_cost.shape)
dc_cust_total_transport_cost.to_csv('extracted_data/34_dc_cust_total_transport_cost.csv') # extract csv file


# throughput of each dc
dc_total_throughput = dc_cust_total_transport_cost.groupby('dc')['quantity'].sum() # add total quantity for each dc
dc_total_throughput = pd.DataFrame(dc_total_throughput) # convert to dataframe
dc_total_throughput.to_csv('extracted_data/35_dc_total_throughput.csv') # extract csv file


# total transportation cost for each dc
dc_total_transport_cost = dc_cust_total_transport_cost.groupby('dc')['total_trans_cost'].sum() # add total transporation cost for each dc
dc_total_transport_cost = pd.DataFrame(dc_total_transport_cost) # convert to dataframe
dc_total_transport_cost.to_csv('extracted_data/36_dc_total_transport_cost.csv') # extract csv file


# total variable cost for each dc
var_cost_idx = var_cost.set_index('dc') # set index 
dc_total_var_cost = var_cost_idx.join(dc_total_throughput) # merge variable cost with througput of each dc 
dc_total_var_cost['total_var_cost'] = dc_total_var_cost['variable_cost'] * dc_total_var_cost['quantity'] # calculate total variable cost based on unit varible cost * total quantity
dc_total_var_cost.drop(columns='quantity', inplace=True) # drop quantity column
dc_total_var_cost.to_csv('extracted_data/37_dc_total_var_cost.csv') # extract csv file

# total network cost for each dc
## summarising the table with all cost values and throughput
fixed_cost_idx = fixed_cost.set_index('dc') # set index 
dc_total_network_cost = fixed_cost_idx.join(dc_total_var_cost) # merge fixed and variable costs
dc_total_network_cost = dc_total_network_cost.join(dc_total_transport_cost) # merging transportation costs
dc_total_network_cost.to_csv('extracted_data/38_dc_total_network_cost.csv') # extract csv file
print(f'Time taken: {float((time.time() - since)/60): 0.2f} in minutes.')

In [ ]:
since = time.time()
# merge customer demand dataframe with sku dataframe and then on dc_cust_locations
cust_demand_idx = cust_demand.set_index('sku')
sku_wt_vol_idx = sku_wt_vol.set_index('sku')
x = pd.merge(cust_demand_idx, sku_wt_vol_idx, on='sku', how = 'left')  # merging 2 dataframes
x.dropna(inplace=True) # remove null values
x.reset_index(inplace=True) # resetting index
x.set_index('cust', inplace=True) # setting index
dc_cust_sku_to_bin = pd.merge(x, dc_cust_mf02, on='cust', how = 'left')  # merging 2 dataframes
dc_cust_sku_to_bin = dc_cust_sku_to_bin[['dc', 'cust', 'sku', 'quantity', 'weight', 'volume', 'mf_01', 'mf_02']]
print('check null values in dc_cust_sku_to_bin')
print(pd.isna(dc_cust_sku_to_bin).sum())

# Bin customer demand dataframe in 120 groups
filter1 = dc_cust_sku_to_bin['quantity']<=10
filter2 = (dc_cust_sku_to_bin['quantity']>10) & (dc_cust_sku_to_bin['quantity']<=60)
filter3 = (dc_cust_sku_to_bin['quantity']>60) & (dc_cust_sku_to_bin['quantity']<=200)
filter4 = (dc_cust_sku_to_bin['quantity']>200) & (dc_cust_sku_to_bin['quantity']<=1000)
filter5 = (dc_cust_sku_to_bin['quantity']>1000) & (dc_cust_sku_to_bin['quantity']<=5000)
filter6 = (dc_cust_sku_to_bin['quantity']>5000) & (dc_cust_sku_to_bin['quantity']<=10000)
filter7 = dc_cust_sku_to_bin['quantity']>10000

labels1 = np.arange(1,11)
labels2 = np.arange(11,21)
labels3 = np.arange(21,31)
labels4 = np.arange(31,81)
labels5 = np.arange(81,106)
labels6 = np.arange(106,116)
labels7 = np.arange(116,121)

x1 = dc_cust_sku_to_bin[filter1]
x2 = dc_cust_sku_to_bin[filter2]
x3 = dc_cust_sku_to_bin[filter3]
x4 = dc_cust_sku_to_bin[filter4]
x5 = dc_cust_sku_to_bin[filter5]
x6 = dc_cust_sku_to_bin[filter6]
x7 = dc_cust_sku_to_bin[filter7]

x1['bin_id'] = pd.cut(x1['quantity'], bins=10, labels=labels1)
x2['bin_id'] = pd.cut(x2['quantity'], bins=10, labels=labels2)
x3['bin_id'] = pd.cut(x3['quantity'], bins=10, labels=labels3)
x4['bin_id'] = pd.cut(x4['quantity'], bins=50, labels=labels4)
x5['bin_id'] = pd.cut(x5['quantity'], bins=25, labels=labels5)
x6['bin_id'] = pd.cut(x6['quantity'], bins=10, labels=labels6)
x7['bin_id'] = pd.cut(x7['quantity'], bins=5, labels=labels7)

print(f'Unique bin counts of quantity <=10 : ', len(x1['bin_id'].unique()))
print(f'Unique bin counts of quantity > 10 but <= 60 : ', len(x2['bin_id'].unique()))
print(f'Unique bin counts of quantity > 60 but <= 20dc_cust_sku_binned0 : ', len(x3['bin_id'].unique()))
print(f'Unique bin counts of quantity > 200 but <= 1000 : ', len(x4['bin_id'].unique()))
print(f'Unique bin counts of quantity > 1000 but <= 5000 : ', len(x5['bin_id'].unique()))
print(f'Unique bin counts of quantity > 5000 but <= 10000 : ', len(x6['bin_id'].unique()))
print(f'Unique bin counts of quantity > 10000: ', len(x7['bin_id'].unique()))

# merge all these splitted dataframes to make a new dataframe - dc_cust_sku_binned
x = [x1, x2, x3, x4, x5, x6, x7]
dc_cust_sku_binned = pd.concat(x)
print('Shape of new dc_cust_sku_binned dataframe: ', dc_cust_sku_binned.shape)
print(f'Unique bin counts of quantity in new dc_cust_sku_binned dataframe: ', len(dc_cust_sku_binned['bin_id'].unique()))


# finding square root(average weight times volume) based on bins
dc_cust_sku_binned['mf_01'] = dc_cust_sku_binned['weight'] * dc_cust_sku_binned['volume'] # weight times volume
average_wt_volume = dc_cust_sku_binned.groupby('bin_id')['mf_01'].mean() # average of weight times volume based on bin_id
average_wt_volume = pd.DataFrame(average_wt_volume) # create dataframe
dc_cust_sku_binned.set_index('bin_id', inplace=True) # set index
dc_cust_sku_binned = pd.merge(dc_cust_sku_binned, average_wt_volume, on='bin_id', how='left') # merge dataframes
dc_cust_sku_binned.rename(columns={'mf_01_x': 'weight_volume', 'mf_01_y': 'avr_weight_volume'}, inplace=True) # rename columns
dc_cust_sku_binned['mf_01'] = np.sqrt(dc_cust_sku_binned['avr_weight_volume']) / 20060 # find the square root(wt*vol) /20060


# finding minimum unit transportation cost
dc_cust_sku_binned['unit_trans_cost'] = dc_cust_sku_binned['mf_01'] * dc_cust_sku_binned['mf_02']
dc_cust_sku_unit_trans_cost = dc_cust_sku_binned.groupby(['cust', 'sku'])['unit_trans_cost'].min()


# calculating total transportation cost for each product for each customer
dc_cust_sku_unit_trans_cost = pd.DataFrame(dc_cust_sku_unit_trans_cost)
dc_cust_sku_total_trans_cost = pd.merge(dc_cust_sku_unit_trans_cost, dc_cust_sku_binned, on=['cust', 'sku', 'unit_trans_cost'], how='left')
dc_cust_sku_total_trans_cost.dropna(inplace=True)
dc_cust_sku_total_trans_cost = dc_cust_sku_total_trans_cost[['dc', 'cust', 'sku', 'quantity', 'weight', 'volume', 'avr_weight_volume' , 'unit_trans_cost']]
dc_cust_sku_total_trans_cost['total_trans_cost'] = dc_cust_sku_total_trans_cost['quantity'] * dc_cust_sku_total_trans_cost['unit_trans_cost']
dc_cust_sku_total_trans_cost.to_csv('extracted_data/41_dc_cust_sku_total_trans_cost.csv') # extract csv file


# throughput of each dc
dc_total_throughput = dc_cust_sku_total_trans_cost.groupby('dc')['quantity'].sum() # add total quantity for each dc
dc_total_throughput = pd.DataFrame(dc_total_throughput) # convert to dataframe
dc_total_throughput.to_csv('extracted_data/42_dc_total_throughput.csv') # extract csv file


# total transportation cost for each dc
dc_total_transport_cost = dc_cust_sku_total_trans_cost.groupby('dc')['total_trans_cost'].sum() # add total transporation cost for each dc
dc_total_transport_cost = pd.DataFrame(dc_total_transport_cost) # convert to dataframe
dc_total_transport_cost.to_csv('extracted_data/43_dc_total_transport_cost.csv') # extract csv file


# total variable cost for each dc
var_cost_idx = var_cost.set_index('dc') # set index 
dc_total_var_cost = var_cost_idx.join(dc_total_throughput) # merge variable cost with througput of each dc 
dc_total_var_cost['total_var_cost'] = dc_total_var_cost['variable_cost'] * dc_total_var_cost['quantity'] # calculate total variable cost based on unit varible cost * total quantity
dc_total_var_cost.drop(columns='quantity', inplace=True) # drop quantity column
dc_total_var_cost.to_csv('extracted_data/44_dc_total_var_cost.csv') # extract csv file

# total network cost for each dc
## summarising the table with all cost values and throughput
fixed_cost_idx = fixed_cost.set_index('dc') # set index 
dc_total_network_cost = fixed_cost_idx.join(dc_total_var_cost) # merge fixed and variable costs
dc_total_network_cost = dc_total_network_cost.join(dc_total_transport_cost) # merging transportation costs
dc_total_network_cost.to_csv('extracted_data/45_dc_total_network_cost.csv') # extract csv file
print(f'Time taken: {float((time.time() - since)/60): 0.2f} in minutes.')

In [ ]:
since = time.time()
# merge customer demand dataframe with sku dataframe and then on dc_cust_locations
cust_demand_idx = cust_demand.set_index('sku')
sku_wt_vol_idx = sku_wt_vol.set_index('sku')
x = pd.merge(cust_demand_idx, sku_wt_vol_idx, on='sku', how = 'left')  # merging 2 dataframes
x.dropna(inplace=True) # remove null values
x.reset_index(inplace=True) # resetting index
x.set_index('cust', inplace=True) # setting index
dc_cust_sku_to_bin = pd.merge(x, dc_cust_mf02, on='cust', how = 'left')  # merging 2 dataframes
dc_cust_sku_to_bin = dc_cust_sku_to_bin[['dc', 'cust', 'sku', 'quantity', 'weight', 'volume', 'mf_01', 'mf_02']]
print('check null values in dc_cust_sku_to_bin')
print(pd.isna(dc_cust_sku_to_bin).sum())

# Bin customer demand dataframe in 120 groups
filter1 = dc_cust_sku_to_bin['quantity']<=10000
filter2 = dc_cust_sku_to_bin['quantity']>10000

labels1 = np.arange(1,11)
labels2 = np.arange(11,13)

x1 = dc_cust_sku_to_bin[filter1]
x2 = dc_cust_sku_to_bin[filter2]

x1['bin_id'] = pd.cut(x1['quantity'], bins=10, labels=labels1)
x2['bin_id'] = pd.cut(x2['quantity'], bins=2, labels=labels2)

print(f'Unique bin counts of quantity <=10000 : ', len(x1['bin_id'].unique()))
print(f'Unique bin counts of quantity > 10000: ', len(x2['bin_id'].unique()))

# merge all these splitted dataframes to make a new dataframe - dc_cust_sku_binned
x = [x1, x2]
dc_cust_sku_binned = pd.concat(x)
print('Shape of new dc_cust_sku_binned dataframe: ', dc_cust_sku_binned.shape)
print(f'Unique bin counts of quantity in new dc_cust_sku_binned dataframe: ', len(dc_cust_sku_binned['bin_id'].unique()))


# finding square root(average weight times volume) based on bins
dc_cust_sku_binned['mf_01'] = dc_cust_sku_binned['weight'] * dc_cust_sku_binned['volume'] # weight times volume
average_wt_volume = dc_cust_sku_binned.groupby('bin_id')['mf_01'].mean() # average of weight times volume based on bin_id
average_wt_volume = pd.DataFrame(average_wt_volume) # create dataframe
dc_cust_sku_binned.set_index('bin_id', inplace=True) # set index
dc_cust_sku_binned = pd.merge(dc_cust_sku_binned, average_wt_volume, on='bin_id', how='left') # merge dataframes
dc_cust_sku_binned.rename(columns={'mf_01_x': 'weight_volume', 'mf_01_y': 'avr_weight_volume'}, inplace=True) # rename columns
dc_cust_sku_binned['mf_01'] = np.sqrt(dc_cust_sku_binned['avr_weight_volume']) / 20060 # find the square root(wt*vol) /20060


# finding minimum unit transportation cost
dc_cust_sku_binned['unit_trans_cost'] = dc_cust_sku_binned['mf_01'] * dc_cust_sku_binned['mf_02']
dc_cust_sku_unit_trans_cost = dc_cust_sku_binned.groupby(['cust', 'sku'])['unit_trans_cost'].min()


# calculating total transportation cost for each product for each customer
dc_cust_sku_unit_trans_cost = pd.DataFrame(dc_cust_sku_unit_trans_cost)
dc_cust_sku_total_trans_cost = pd.merge(dc_cust_sku_unit_trans_cost, dc_cust_sku_binned, on=['cust', 'sku', 'unit_trans_cost'], how='left')
dc_cust_sku_total_trans_cost.dropna(inplace=True)
dc_cust_sku_total_trans_cost = dc_cust_sku_total_trans_cost[['dc', 'cust', 'sku', 'quantity', 'weight', 'volume', 'avr_weight_volume' , 'unit_trans_cost']]
dc_cust_sku_total_trans_cost['total_trans_cost'] = dc_cust_sku_total_trans_cost['quantity'] * dc_cust_sku_total_trans_cost['unit_trans_cost']
dc_cust_sku_total_trans_cost.to_csv('extracted_data/51_dc_cust_sku_total_trans_cost.csv') # extract csv file


# throughput of each dc
dc_total_throughput = dc_cust_sku_total_trans_cost.groupby('dc')['quantity'].sum() # add total quantity for each dc
dc_total_throughput = pd.DataFrame(dc_total_throughput) # convert to dataframe
dc_total_throughput.to_csv('extracted_data/52_dc_total_throughput.csv') # extract csv file


# total transportation cost for each dc
dc_total_transport_cost = dc_cust_sku_total_trans_cost.groupby('dc')['total_trans_cost'].sum() # add total transporation cost for each dc
dc_total_transport_cost = pd.DataFrame(dc_total_transport_cost) # convert to dataframe
dc_total_transport_cost.to_csv('extracted_data/53_dc_total_transport_cost.csv') # extract csv file


# total variable cost for each dc
var_cost_idx = var_cost.set_index('dc') # set index 
dc_total_var_cost = var_cost_idx.join(dc_total_throughput) # merge variable cost with througput of each dc 
dc_total_var_cost['total_var_cost'] = dc_total_var_cost['variable_cost'] * dc_total_var_cost['quantity'] # calculate total variable cost based on unit varible cost * total quantity
dc_total_var_cost.drop(columns='quantity', inplace=True) # drop quantity column
dc_total_var_cost.to_csv('extracted_data/54_dc_total_var_cost.csv') # extract csv file

# total network cost for each dc
## summarising the table with all cost values and throughput
fixed_cost_idx = fixed_cost.set_index('dc') # set index 
dc_total_network_cost = fixed_cost_idx.join(dc_total_var_cost) # merge fixed and variable costs
dc_total_network_cost = dc_total_network_cost.join(dc_total_transport_cost) # merging transportation costs
dc_total_network_cost.to_csv('extracted_data/55_dc_total_network_cost.csv') # extract csv file
print(f'Time taken: {float((time.time() - since)/60): 0.2f} in minutes.')